In [1]:
# Malta Traffic Accident Analysis - Logistic Regression
# ICS5110 Applied Machine Learning Assignment
# Student: Naomi Thornley & Giulia-Maria Montebonello
# Date: January 2026

"""
This notebook implements Logistic Regression to predict accident severity.

Research Questions Addressed:
RQ1: How accurately can ML predict minor vs severe injuries?
     → Measure accuracy, precision, recall, F1-score
RQ2: Which features matter most for predicting severity?
     → Analyze logistic regression coefficients
RQ3: Does motorcycle involvement increase severity?
     → Check motorcycle coefficient significance

Our approach:
1. Prepare the data (encode categories, scale features)
2. Split into train/test (80/20)
3. Train logistic regression
4. Evaluate performance
5. Interpret coefficients to answer RQs
"""

# IMPORT LIBRARIES

print("Loading all the packages I need...")

# For working with data
import pandas as pd
import numpy as np

# For machine learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score, roc_curve
)

# For making charts
import matplotlib.pyplot as plt
import seaborn as sns

# Settings to make output cleaner
import warnings
warnings.filterwarnings('ignore')  # Hide warning messages

pd.set_option('display.max_columns', None)  # Show all columns
%matplotlib inline  

print("✅ All packages loaded successfully!")

Loading all the packages I need...
✅ All packages loaded successfully!


In [2]:
# LOAD THE CLEAN DATA

print("="*70)
print("LOADING MY ML-READY DATA")
print("="*70)

# Load the dataset I prepared in Notebook 1
df = pd.read_csv('data/processed/accidents_ml_ready.csv')

print(f"\n✅ Data loaded!")
print(f"   I have {len(df)} accident records")
print(f"   With {len(df.columns)} columns")

# Let's see what I'm trying to predict
print(f"\n📊 What I'm predicting (severity_binary):")
print(df['severity_binary'].value_counts())

high_count = (df['severity_binary'] == 'high').sum()
low_count = (df['severity_binary'] == 'low').sum()
print(f"\n   High severity (fatal/grievous): {high_count} ({high_count/len(df)*100:.1f}%)")
print(f"   Low severity (serious/slight): {low_count} ({low_count/len(df)*100:.1f}%)")

LOADING MY ML-READY DATA

✅ Data loaded!
   I have 318 accident records
   With 33 columns

📊 What I'm predicting (severity_binary):
severity_binary
high    251
low      67
Name: count, dtype: int64

   High severity (fatal/grievous): 251 (78.9%)
   Low severity (serious/slight): 67 (21.1%)


In [3]:
# DECIDING WHICH FEATURES TO USE

print("="*70)
print("SELECTING FEATURES FOR MY MODEL")
print("="*70)

print("\n🤔 Which features should I include?")
print("   I need features that might predict severity AND answer my research questions!")

# Features I'll use - grouped by type
temporal_features = ['month', 'is_weekend', 'is_holiday', 'has_time']
location_features = ['has_location']
weather_features = ['temperature', 'precipitation', 'wind_speed', 'is_rainy', 'is_windy']
vehicle_features = ['has_motorcycle']  # Important for RQ3!

# Categorical features (need to encode these as numbers)
categorical_features = [
    'day_of_week',      # Monday, Tuesday, etc.
    'region',           # Malta vs Gozo
    'weather_condition',# Clear, rain, etc.
    'vehicle_category', # Motorcycle involved, car only, etc.
    'time_of_day',      # Morning, afternoon, etc.
    'season',           # Summer, winter, etc.
    'area_type'         # Urban vs rural
]

print(f"\n📋 My feature groups:")
print(f"   ⏰ Temporal: {temporal_features}")
print(f"   📍 Location: {location_features}")
print(f"   🌦️ Weather: {weather_features}")
print(f"   🚗 Vehicle: {vehicle_features}")
print(f"   📝 Categorical (need encoding): {categorical_features}")

# Count total features
numerical_features = temporal_features + location_features + weather_features + vehicle_features
total_features = len(numerical_features) + len(categorical_features)

print(f"\n✅ I'm starting with {total_features} features")
print(f"   ({len(numerical_features)} numerical + {len(categorical_features)} categorical)")

SELECTING FEATURES FOR MY MODEL

🤔 Which features should I include?
   I need features that might predict severity AND answer my research questions!

📋 My feature groups:
   ⏰ Temporal: ['month', 'is_weekend', 'is_holiday', 'has_time']
   📍 Location: ['has_location']
   🌦️ Weather: ['temperature', 'precipitation', 'wind_speed', 'is_rainy', 'is_windy']
   🚗 Vehicle: ['has_motorcycle']
   📝 Categorical (need encoding): ['day_of_week', 'region', 'weather_condition', 'vehicle_category', 'time_of_day', 'season', 'area_type']

✅ I'm starting with 18 features
   (11 numerical + 7 categorical)


In [4]:
# =============================================================================
# CONVERTING CATEGORIES TO NUMBERS
# =============================================================================
print("="*70)
print("ENCODING CATEGORICAL FEATURES")
print("="*70)

print("\n🔄 Problem: ML models need numbers, not text!")
print("   'Monday', 'Tuesday' → need to become numbers")
print("   Solution: One-hot encoding (create binary columns)")

# Make a copy so I don't mess up my original data
df_model = df.copy()

# ALL categorical features that need encoding (COMPLETE LIST)
categorical_features = [
    'day_of_week',       # Monday, Tuesday, etc.
    'region',            # Malta vs Gozo
    'weather_condition', # Clear, rain, etc.
    'vehicle_category',  # Motorcycle involved, car only, etc.
    'time_of_day',       # Morning, afternoon, etc.
    'season',            # Summer, winter, etc.
    'area_type',         # Urban vs rural
    'hour_category'      # Rush hour categories - WE MISSED THIS!
]

print(f"\n📋 Categorical features to encode:")
for feat in categorical_features:
    unique_vals = df_model[feat].nunique()
    print(f"   {feat}: {unique_vals} unique values")

# One-hot encode the categorical features
# drop_first=True avoids multicollinearity (important for logistic regression!)
df_encoded = pd.get_dummies(df_model, columns=categorical_features, drop_first=True)

print(f"\n✅ Encoding done!")
print(f"   Before encoding: {len(df_model.columns)} columns")
print(f"   After encoding: {len(df_encoded.columns)} columns")

# Figure out which columns are my features (exclude metadata and target)
columns_to_exclude = [
    'title', 'content', 'date', 'source', 'time', 
    'severity', 'severity_binary', 'severity_3class',  # Target variables
    'vehicles', 'location', 'weather_code',  # Text columns
    'year', 'hour', 'month_name', 'date_modified'  # Not using these directly
]

# Get all columns that aren't excluded
feature_columns = [col for col in df_encoded.columns if col not in columns_to_exclude]

print(f"\n✅ After encoding, I have {len(feature_columns)} features for modeling")

print(f"\n📋 Here are all my features:")
for i, feat in enumerate(feature_columns, 1):
    print(f"   {i:2d}. {feat}")

ENCODING CATEGORICAL FEATURES

🔄 Problem: ML models need numbers, not text!
   'Monday', 'Tuesday' → need to become numbers
   Solution: One-hot encoding (create binary columns)

📋 Categorical features to encode:
   day_of_week: 7 unique values
   region: 3 unique values
   weather_condition: 3 unique values
   vehicle_category: 3 unique values
   time_of_day: 5 unique values
   season: 3 unique values
   area_type: 3 unique values
   hour_category: 5 unique values

✅ Encoding done!
   Before encoding: 33 columns
   After encoding: 49 columns

✅ After encoding, I have 36 features for modeling

📋 Here are all my features:
    1. month
    2. is_weekend
    3. has_time
    4. has_location
    5. has_motorcycle
    6. is_holiday
    7. temperature
    8. precipitation
    9. wind_speed
   10. is_rainy
   11. is_foggy
   12. is_windy
   13. day_of_week_Monday
   14. day_of_week_Saturday
   15. day_of_week_Sunday
   16. day_of_week_Thursday
   17. day_of_week_Tuesday
   18. day_of_week_Wedn

In [5]:
# PREPARING X (FEATURES) AND y (TARGET)

print("="*70)
print("SETTING UP X AND y")
print("="*70)

print("\n📊 In ML, we call:")
print("   X = features (what I use to predict)")
print("   y = target (what I'm trying to predict)")

# X = all my features
X = df_encoded[feature_columns]

# y = target variable (severity)
# Convert 'high'/'low' to 1/0 for the model
y = (df_encoded['severity_binary'] == 'high').astype(int)
# Now: 1 = high severity, 0 = low severity

print(f"\n✅ X (features) shape: {X.shape}")
print(f"   → {X.shape[0]} accidents (rows)")
print(f"   → {X.shape[1]} features (columns)")

print(f"\n✅ y (target) shape: {y.shape}")
print(f"   Distribution:")
print(f"   → Class 1 (high severity): {y.sum()} accidents ({y.sum()/len(y)*100:.1f}%)")
print(f"   → Class 0 (low severity): {(y == 0).sum()} accidents ({(y == 0).sum()/len(y)*100:.1f}%)")

# Quick check for missing values
print(f"\n🔍 Checking for missing values...")
missing_count = X.isnull().sum().sum()

if missing_count > 0:
    print(f"   ⚠️ Found {missing_count} missing values")
    print(f"   Filling them with 0 (simple approach)")
    X = X.fillna(0)
    print(f"   ✅ Done!")
else:
    print(f"   ✅ No missing values - perfect!")

SETTING UP X AND y

📊 In ML, we call:
   X = features (what I use to predict)
   y = target (what I'm trying to predict)

✅ X (features) shape: (318, 36)
   → 318 accidents (rows)
   → 36 features (columns)

✅ y (target) shape: (318,)
   Distribution:
   → Class 1 (high severity): 251 accidents (78.9%)
   → Class 0 (low severity): 67 accidents (21.1%)

🔍 Checking for missing values...
   ✅ No missing values - perfect!


In [6]:
# SPLITTING INTO TRAINING AND TEST SETS

print("="*70)
print("SPLITTING DATA: TRAIN SET vs TEST SET")
print("="*70)

print("\n📚 Why split?")
print("   → Train set: Model learns from this (80% of data)")
print("   → Test set: Model is evaluated on this (20% of data)")
print("   → This way I can see if the model generalizes to new data!")

# Split the data
# test_size=0.2 means 20% for testing
# random_state=42 makes it reproducible (same split every time)
# stratify=y keeps the same class balance in both sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"\n✅ Split complete!")

print(f"\n📊 Training set (model will learn from this):")
print(f"   → {len(X_train)} accidents")
print(f"   → High severity: {y_train.sum()} ({y_train.sum()/len(y_train)*100:.1f}%)")
print(f"   → Low severity: {(y_train == 0).sum()} ({(y_train == 0).sum()/len(y_train)*100:.1f}%)")

print(f"\n📊 Test set (will test the model on this):")
print(f"   → {len(X_test)} accidents")
print(f"   → High severity: {y_test.sum()} ({y_test.sum()/len(y_test)*100:.1f}%)")
print(f"   → Low severity: {(y_test == 0).sum()} ({(y_test == 0).sum()/len(y_test)*100:.1f}%)")

SPLITTING DATA: TRAIN SET vs TEST SET

📚 Why split?
   → Train set: Model learns from this (80% of data)
   → Test set: Model is evaluated on this (20% of data)
   → This way I can see if the model generalizes to new data!

✅ Split complete!

📊 Training set (model will learn from this):
   → 254 accidents
   → High severity: 200 (78.7%)
   → Low severity: 54 (21.3%)

📊 Test set (will test the model on this):
   → 64 accidents
   → High severity: 51 (79.7%)
   → Low severity: 13 (20.3%)


In [7]:
# SCALING FEATURES 

print("="*70)
print("SCALING FEATURES TO THE SAME RANGE")
print("="*70)

print("\n🤔 Why do I need to scale?")
print("   Problem: My features are on very different scales:")
print("   → temperature: 12-30 (range of ~18)")
print("   → wind_speed: 6-49 (range of ~43)")
print("   → is_weekend: 0-1 (range of 1)")
print("\n   Logistic regression is sensitive to scale!")
print("   → Features with bigger ranges would dominate")
print("   → Scaling puts everything on the same scale")

print("\n🔧 Solution: StandardScaler")
print("   Transforms each feature to: mean=0, std=1")

# Create the scaler
scaler = StandardScaler()

# IMPORTANT: Fit the scaler on training data only!
# Then use it to transform both train and test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame so I keep the column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print(f"\n✅ Scaling complete!")

# Show an example to prove it worked
temp_col = [col for col in X_train.columns if 'temperature' in col.lower()]
if temp_col:
    col_name = temp_col[0]
    print(f"\n📊 Example: {col_name}")
    print(f"   Before scaling: mean={X_train[col_name].mean():.2f}, std={X_train[col_name].std():.2f}")
    print(f"   After scaling:  mean={X_train_scaled[col_name].mean():.2f}, std={X_train_scaled[col_name].std():.2f}")
    print(f"   → Now it's standardized (mean≈0, std≈1)!")
else:
    print(f"\n   Features are now standardized (mean≈0, std≈1)")

SCALING FEATURES TO THE SAME RANGE

🤔 Why do I need to scale?
   Problem: My features are on very different scales:
   → temperature: 12-30 (range of ~18)
   → wind_speed: 6-49 (range of ~43)
   → is_weekend: 0-1 (range of 1)

   Logistic regression is sensitive to scale!
   → Features with bigger ranges would dominate
   → Scaling puts everything on the same scale

🔧 Solution: StandardScaler
   Transforms each feature to: mean=0, std=1

✅ Scaling complete!

📊 Example: temperature
   Before scaling: mean=21.06, std=5.46
   After scaling:  mean=-0.00, std=1.00
   → Now it's standardized (mean≈0, std≈1)!


In [8]:
# TRAINING LOGISTIC REGRESSION MODEL

print("="*70)
print("TRAINING LOGISTIC REGRESSION MODEL")
print("="*70)

print("\n🎯 Model: Logistic Regression")
print("   Why this model?")
print("   ✓ Good for binary classification (high vs low severity)")
print("   ✓ Fast to train")
print("   ✓ Interpretable - I can see which features matter!")
print("   ✓ Gives me coefficients to answer RQ2 and RQ3")

print("\n🔧 Setting up the model...")

# Create the logistic regression model
# max_iter=1000 to make sure it converges
# random_state=42 for reproducibility
model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs'  # Good general-purpose solver
)

print(f"✅ Model created!")
print(f"   Parameters: {model.get_params()}")

print("\n🏋️ Training the model on {len(X_train_scaled)} accidents...")
print("   (This might take a few seconds...)")

# Train the model
model.fit(X_train_scaled, y_train)

print(f"\n✅ Training complete!")
print(f"   The model has learned from the training data!")
print(f"   Number of iterations: {model.n_iter_[0]}")

print("\n💡 Now let's see how well it performs!")

TRAINING LOGISTIC REGRESSION MODEL

🎯 Model: Logistic Regression
   Why this model?
   ✓ Good for binary classification (high vs low severity)
   ✓ Fast to train
   ✓ Interpretable - I can see which features matter!
   ✓ Gives me coefficients to answer RQ2 and RQ3

🔧 Setting up the model...
✅ Model created!
   Parameters: {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': 0.0, 'max_iter': 1000, 'n_jobs': None, 'penalty': 'deprecated', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}

🏋️ Training the model on {len(X_train_scaled)} accidents...
   (This might take a few seconds...)

✅ Training complete!
   The model has learned from the training data!
   Number of iterations: 20

💡 Now let's see how well it performs!


In [9]:
# MAKING PREDICTIONS

print("="*70)
print("MAKING PREDICTIONS ON TEST SET")
print("="*70)

print("\n🔮 Time to test the model!")
print(f"   The model has NEVER seen these {len(X_test_scaled)} accidents before")
print(f"   Let's see if it can predict their severity correctly!")

# Make predictions on test set
y_pred = model.predict(X_test_scaled)

# Also get prediction probabilities (useful for ROC curve later)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]  # Probability of class 1 (high severity)

print(f"\n✅ Predictions made!")

# Quick look at predictions
print(f"\n📊 Prediction summary:")
print(f"   Predicted HIGH severity: {(y_pred == 1).sum()} accidents")
print(f"   Predicted LOW severity: {(y_pred == 0).sum()} accidents")

print(f"\n📊 Actual values in test set:")
print(f"   Actually HIGH severity: {y_test.sum()} accidents")
print(f"   Actually LOW severity: {(y_test == 0).sum()} accidents")

print(f"\n💡 Now let's calculate how accurate these predictions are!")

MAKING PREDICTIONS ON TEST SET

🔮 Time to test the model!
   The model has NEVER seen these 64 accidents before
   Let's see if it can predict their severity correctly!

✅ Predictions made!

📊 Prediction summary:
   Predicted HIGH severity: 61 accidents
   Predicted LOW severity: 3 accidents

📊 Actual values in test set:
   Actually HIGH severity: 51 accidents
   Actually LOW severity: 13 accidents

💡 Now let's calculate how accurate these predictions are!


In [10]:
# EVALUATING MODEL PERFORMANCE 

print("="*70)
print("MODEL PERFORMANCE EVALUATION")
print("="*70)

print("\n🎯 RQ1: How accurately can ML predict minor vs severe injuries?")
print("   Let's find out!")

# Calculate performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"\n📊 PERFORMANCE METRICS:")
print(f"="*70)

print(f"\n✅ ACCURACY: {accuracy:.3f} ({accuracy*100:.1f}%)")
print(f"   → Out of 100 predictions, {accuracy*100:.0f} are correct")
print(f"   → This directly answers RQ1!")

print(f"\n✅ PRECISION: {precision:.3f} ({precision*100:.1f}%)")
print(f"   → When I predict 'high severity', I'm right {precision*100:.0f}% of the time")
print(f"   → Important for resource allocation!")

print(f"\n✅ RECALL: {recall:.3f} ({recall*100:.1f}%)")
print(f"   → I correctly identify {recall*100:.0f}% of actual high severity accidents")
print(f"   → Important for safety - don't want to miss severe accidents!")

print(f"\n✅ F1-SCORE: {f1:.3f}")
print(f"   → Balance between precision and recall")
print(f"   → Good overall performance indicator")

print(f"\n✅ ROC-AUC: {roc_auc:.3f}")
print(f"   → Measures model's ability to discriminate between classes")
print(f"   → 0.5 = random, 1.0 = perfect")
print(f"   → {roc_auc:.3f} is {'excellent!' if roc_auc > 0.8 else 'good!' if roc_auc > 0.7 else 'okay'}")

# Detailed classification report
print(f"\n📋 DETAILED CLASSIFICATION REPORT:")
print(f"="*70)
print(classification_report(y_test, y_pred, target_names=['Low Severity', 'High Severity']))

print(f"\n🎯 ANSWER TO RQ1:")
print(f"   Logistic Regression can predict accident severity with")
print(f"   {accuracy*100:.1f}% accuracy!")

MODEL PERFORMANCE EVALUATION

🎯 RQ1: How accurately can ML predict minor vs severe injuries?
   Let's find out!

📊 PERFORMANCE METRICS:

✅ ACCURACY: 0.781 (78.1%)
   → Out of 100 predictions, 78 are correct
   → This directly answers RQ1!

✅ PRECISION: 0.803 (80.3%)
   → When I predict 'high severity', I'm right 80% of the time
   → Important for resource allocation!

✅ RECALL: 0.961 (96.1%)
   → I correctly identify 96% of actual high severity accidents
   → Important for safety - don't want to miss severe accidents!

✅ F1-SCORE: 0.875
   → Balance between precision and recall
   → Good overall performance indicator

✅ ROC-AUC: 0.520
   → Measures model's ability to discriminate between classes
   → 0.5 = random, 1.0 = perfect
   → 0.520 is okay

📋 DETAILED CLASSIFICATION REPORT:
               precision    recall  f1-score   support

 Low Severity       0.33      0.08      0.12        13
High Severity       0.80      0.96      0.88        51

     accuracy                           0